In [1]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)

unk/embed = 4415/15107

# CNN

In [2]:
from main import Config
from data_utils import SentDatasetReader, BucketIterator

config = Config(model_name='cnn', include_length=False)
reader = SentDatasetReader(config)

train_set, val_set, test_set = reader.train_set, reader.val_set, reader.test_set
print(len(train_set), len(val_set), len(test_set))

tokenizer = reader.tokenizer

train_iter = BucketIterator(config, train_set[:50], tokenizer)
print(len(train_iter))

INFO:transformers.file_utils:PyTorch version 1.3.0+cpu available.
INFO:data_utils:Reading TRAINING set from data/hotel\train.csv
INFO:data_utils:Loading vocab & embedding from caches/hotel_tokenizer.pt
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.688 seconds.
DEBUG:jieba:Loading model cost 1.688 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
INFO:data_utils:Read 3500 lines from data/hotel\train.csv
INFO:data_utils:Reading VALIDATION set from data/hotel\val.csv
INFO:data_utils:Read 500 lines from data/hotel\val.csv
INFO:data_utils:Reading TEST set from data/hotel\test.csv
INFO:data_utils:Read 800 lines from data/hotel\test.csv


3500 500 800
1


In [9]:
from models import TextCNN
import torch.optim as optim
import torch.nn as nn

model = TextCNN(tokenizer.embedding, tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    for batch in train_iter:
        optimizer.zero_grad()
        text, label = batch['text'], batch['label']
        pred = model(text)
        loss = criterion(pred, label)
        loss.backward()
        print(loss.item())
        optimizer.step()


0.7392774224281311
0.49718213081359863
0.3517269492149353
0.28869447112083435
0.18163490295410156
0.14854934811592102
0.11790812760591507
0.08855190873146057
0.06789091229438782
0.05303627625107765
0.04093398153781891
0.024127090349793434
0.021641969680786133
0.018726971000432968
0.014116057194769382
0.009684654884040356
0.009819768369197845
0.006484685465693474
0.005935234483331442
0.00454971706494689


# RNN

In [2]:
from main import Config
from data_utils import SentDatasetReader, BucketIterator

config = Config(model_name='rnn', include_length=True)
reader = SentDatasetReader(config)

train_set, val_set, test_set = reader.train_set, reader.val_set, reader.test_set
print(len(train_set), len(val_set), len(test_set))

tokenizer = reader.tokenizer

train_iter = BucketIterator(config, train_set[:80], tokenizer)
print(len(train_iter))

INFO:transformers.file_utils:PyTorch version 1.3.0+cpu available.
INFO:data_utils:Reading TRAINING set from data/hotel\train.csv
INFO:data_utils:Loading vocab & embedding from caches/hotel_tokenizer.pt
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.083 seconds.
DEBUG:jieba:Loading model cost 1.083 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
INFO:data_utils:Read 3500 lines from data/hotel\train.csv
INFO:data_utils:Reading VALIDATION set from data/hotel\val.csv
INFO:data_utils:Read 500 lines from data/hotel\val.csv
INFO:data_utils:Reading TEST set from data/hotel\test.csv
INFO:data_utils:Read 800 lines from data/hotel\test.csv


3500 500 800
2


In [3]:
from models import RNN
import torch.optim as optim
import torch.nn as nn

model = RNN(tokenizer.embedding, tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    for batch in train_iter:
        text, length, label = batch['text'], batch['length'], batch['label']
        optimizer.zero_grad()
        pred = model(text, length)
        loss = criterion(pred, label)
        loss.backward()
        print(loss.item())
        optimizer.step()

0.6889774799346924
0.7397022247314453
0.6576491594314575
0.7153794765472412
0.688977062702179
0.6684532761573792
0.6300277709960938
0.6727902293205261
0.6633433103561401
0.5792187452316284
0.5504500865936279
0.6030893325805664
0.47300875186920166
0.5201376080513
0.34177902340888977
0.4162200391292572
0.37619826197624207
0.30556660890579224
0.28590646386146545
0.10428847372531891
0.3557618260383606
0.07192455232143402
0.055615682154893875
0.449155330657959
0.18784195184707642
0.049717772752046585
0.10942044109106064
0.5472766757011414
0.4109153747558594
0.04546129330992699
0.2228892743587494
0.13001637160778046
0.2464221566915512
0.08700109273195267
0.09485174715518951
0.24580833315849304
0.2056538462638855
0.017403576523065567
0.12352816760540009
0.012450458481907845


# BERT

In [6]:
from main import Config
from data_utils import SentDatasetReader, BucketIterator

config = Config(model_name='bert', include_length=True, max_len=512)
reader = SentDatasetReader(config)

train_set, val_set, test_set = reader.train_set, reader.val_set, reader.test_set
print(len(train_set), len(val_set), len(test_set))

tokenizer = reader.tokenizer

train_iter = BucketIterator(config, train_set[:50], tokenizer)
print(len(train_iter))

INFO:data_utils:Reading TRAINING set from data/hotel\train.csv
INFO:transformers.tokenization_utils:Model name 'hfl/chinese-bert-wwm-ext' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'hfl/chinese-bert-wwm-ext' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/hfl/chinese-bert-wwm-ext/vocab.txt from cache at C:\Users\lenovo\

3500 500 800


ValueError: expected sequence of length 368 at dim 1 (got 366)

In [2]:
x = [1, 3, 5, 7]
print(x[:1000])

[1, 3, 5, 7]
